# STL Model + Hatching Visualization with PyVista

This notebook demonstrates:
1. **Loading STL files** using pyslm
2. **Slicing STL models** with configurable options
3. **Generating hatching paths** with various strategies
4. **Visualizing STL + Hatching** together in PyVista
5. **Color-coding by laser parameters** (Power, Velocity, Energy)

## Features:
- 🎨 **STL Model Visualization** - 3D mesh display
- 🔄 **Hatching Paths** - Generated scan vectors
- ⚡ **Laser Parameters** - Power, Velocity, Energy color-coding
- 🎛️ **Interactive Controls** - Adjust slicing and hatching parameters
- 📊 **Multi-layer Support** - Visualize across build height

## Workflow:
1. Load STL file → `pyslm.Part`
2. Slice with options → `getVectorSlice(z)`
3. Generate hatching → `hatcher.hatch()`
4. Visualize in PyVista → STL mesh + hatching paths
5. Export to .slm (optional) → `libSLM.slmsol.Writer()`


## Setup and Imports


In [1]:
"""
Setup and imports for STL + Hatching Visualization
"""

import sys
from pathlib import Path
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# ============================================================================
# AUTO-ADD amenv PATHS
# ============================================================================
amenv_site_packages = "/home/kgupta/miniconda3/envs/amenv/lib/python3.11/site-packages"
amenv_egg_path = "/home/kgupta/miniconda3/envs/amenv/lib/python3.11/site-packages/PythonSLM-0.6.0-py3.11.egg"

if amenv_site_packages not in sys.path:
    sys.path.insert(0, amenv_site_packages)
    print(f"✅ Added amenv site-packages to path")

if amenv_egg_path not in sys.path:
    sys.path.insert(0, amenv_egg_path)
    print(f"✅ Added amenv egg path to path")

# Import PyVista
try:
    import pyvista as pv
    pv.set_jupyter_backend('static')
    PYVISTA_AVAILABLE = True
    print("✅ PyVista imported successfully")
except ImportError:
    print("⚠️ PyVista not available. Install with: pip install pyvista")
    PYVISTA_AVAILABLE = False

# Import ipywidgets
try:
    import ipywidgets as widgets
    from ipywidgets import HBox, VBox, Output
    from IPython.display import display, clear_output
    WIDGETS_AVAILABLE = True
except ImportError:
    print("⚠️ ipywidgets not available. Install with: pip install ipywidgets")
    WIDGETS_AVAILABLE = False

# Import pyslm
try:
    import pyslm
    import pyslm.geometry
    import pyslm.visualise
    from pyslm import hatching
    PYSLM_AVAILABLE = True
    print("✅ pyslm imported successfully")
except ImportError:
    print("⚠️ pyslm not available.")
    PYSLM_AVAILABLE = False

# Import libSLM (for export)
try:
    from libSLM import slmsol
    LIBSLM_AVAILABLE = True
    print("✅ libSLM imported successfully")
except ImportError:
    print("⚠️ libSLM not available (export functionality disabled)")
    LIBSLM_AVAILABLE = False

print("✅ Setup complete!")
print(f"   - PyVista: {PYVISTA_AVAILABLE}")
print(f"   - pyslm: {PYSLM_AVAILABLE}")
print(f"   - libSLM: {LIBSLM_AVAILABLE}")
print(f"   - ipywidgets: {WIDGETS_AVAILABLE}")


✅ PyVista imported successfully
✅ pyslm imported successfully
✅ libSLM imported successfully
✅ Setup complete!
   - PyVista: True
   - pyslm: True
   - libSLM: True
   - ipywidgets: True


## 1. Load STL File
figures

In [ ]:
# Find STL file
from pathlib import Path
current_dir = Path().resolve()
print(f"🔍 Current directory: {current_dir}")

# Determine project root (go up from demo if we're in demo)
if current_dir.name == "demo":
    project_root = current_dir.parent
else:
    project_root = current_dir

print(f"🔍 Project root: {project_root}")

# Look for models directory - check external/pyslm/models first
models_dir = None
stl_file = None

# Priority 1: External pyslm models folder (src/data_pipeline/external/pyslm/models)
external_models_dir = project_root / "src" / "data_pipeline" / "external" / "pyslm" / "models"
print(f"🔍 Checking external pyslm models: {external_models_dir}")
if external_models_dir.exists():
    print(f"✅ Found external pyslm models directory: {external_models_dir}")
    models_dir = external_models_dir
else:
    print(f"⚠️ External pyslm models directory not found at: {external_models_dir}")
    models_dir = None

# Priority 2: Project root models folder
if models_dir is None:
    models_dir = project_root / "models"
    print(f"🔍 Checking project root models: {models_dir}")
    if models_dir.exists():
        print(f"✅ Found models directory: {models_dir}")
    else:
        print(f"⚠️ Models directory not found at: {models_dir}")
        models_dir = None

# Priority 3: Try pyslm package models folder (installed package)
if models_dir is None and PYSLM_AVAILABLE:
    try:
        import pyslm
        pyslm_path = Path(pyslm.__file__).parent
        models_dir = pyslm_path / "models"
        print(f"🔍 Checking installed pyslm models folder: {models_dir}")
        
        if models_dir.exists():
            print(f"✅ Found pyslm models directory: {models_dir}")
        else:
            print(f"⚠️ pyslm models directory not found at: {models_dir}")
            models_dir = None
    except Exception as e:
        print(f"⚠️ Could not locate pyslm package: {e}")
        models_dir = None

# Priority 4: Try other fallback locations
if models_dir is None:
    print(f"🔍 Trying fallback locations...")
    search_paths = [
        current_dir / "models",
        current_dir.parent / "models",
    ]
    
    for path in search_paths:
        print(f"🔍 Checking: {path}")
        if path.exists():
            models_dir = path
            print(f"✅ Found models directory: {models_dir}")
            break

# Find STL file
if models_dir and models_dir.exists():
    stl_file = models_dir / "frameGuide.stl"
    print(f"🔍 Looking for: {stl_file}")
    if stl_file.exists():
        print(f"📁 Found STL file: {stl_file}")
    else:
        # Try to find any .stl file
        stl_files = list(models_dir.glob("*.stl"))
        if stl_files:
            stl_file = stl_files[0]
            print(f"📁 Found STL file: {stl_file} (not frameGuide.stl, using first available)")
        else:
            print(f"⚠️ No STL files found in {models_dir}")
            print(f"   Directory contents: {list(models_dir.iterdir()) if models_dir.exists() else 'N/A'}")
            stl_file = None
else:
    print(f"⚠️ Models directory not found.")
    print(f"   Expected location: {project_root / 'src' / 'data_pipeline' / 'external' / 'pyslm' / 'models'}")
    print(f"   Or set stl_file_path manually in the code below.")

# Load STL file with pyslm
if stl_file and stl_file.exists() and PYSLM_AVAILABLE:
    print(f"\n📖 Loading STL file: {stl_file}")
    
    try:
        # Create part and load STL
        solidPart = pyslm.Part('frameGuide')
        solidPart.setGeometry(str(stl_file))
        
        # Optional: Transform the part
        solidPart.origin = [0.0, 0.0, 0.0]
        solidPart.rotation = [0, 0, 0]
        solidPart.dropToPlatform()  # Drop to z=0
        
        print(f"✅ STL file loaded successfully!")
        print(f"   - Bounding box: {solidPart.boundingBox}")
        print(f"   - Origin: {solidPart.origin}")
        print(f"   - Rotation: {solidPart.rotation}")
        
        # Store for later use
        stl_part = solidPart
        
    except Exception as e:
        print(f"❌ Error loading STL file: {e}")
        import traceback
        traceback.print_exc()
        stl_part = None
else:
    print("❌ STL file not found or pyslm not available.")
    stl_part = None


🔍 Current directory: /mnt/c/Users/kanha/Independent_Research/pbf-lbm-nosql-data-warehouse/demo
🔍 Project root: /mnt/c/Users/kanha/Independent_Research/pbf-lbm-nosql-data-warehouse
🔍 Checking external pyslm models: /mnt/c/Users/kanha/Independent_Research/pbf-lbm-nosql-data-warehouse/src/data_pipeline/external/pyslm/models
✅ Found external pyslm models directory: /mnt/c/Users/kanha/Independent_Research/pbf-lbm-nosql-data-warehouse/src/data_pipeline/external/pyslm/models
🔍 Looking for: /mnt/c/Users/kanha/Independent_Research/pbf-lbm-nosql-data-warehouse/src/data_pipeline/external/pyslm/models/frameGuide.stl
📁 Found STL file: /mnt/c/Users/kanha/Independent_Research/pbf-lbm-nosql-data-warehouse/src/data_pipeline/external/pyslm/models/frameGuide.stl

📖 Loading STL file: /mnt/c/Users/kanha/Independent_Research/pbf-lbm-nosql-data-warehouse/src/data_pipeline/external/pyslm/models/frameGuide.stl
✅ STL file loaded successfully!
   - Bounding box: [-2.40000000e+01 -5.60000000e+01 -2.37497916e-15  

## 2. Configure Hatching Parameters

Adjust these parameters to control how the STL is sliced and hatched.


In [3]:
# Hatching Configuration (can be adjusted)
HATCHING_CONFIG = {
    'layer_thickness': 0.04,  # mm
    'hatch_angle': 10,  # degrees
    'volume_offset': 0.08,  # mm
    'spot_compensation': 0.06,  # mm
    'num_inner_contours': 2,
    'num_outer_contours': 1,
    'stripe_width': 5.0  # mm
}

# Variable Laser Parameters per Layer (Discrete)
# Format: {layer_range: (power, speed), ...}
# layer_range can be:
#   - int: single layer index
#   - tuple (start, end): layer range [start, end)
#   - 'default': default values for all other layers
LAYER_PARAMETERS = {
    'default': (200.0, 500.0),  # (power W, speed mm/s) - default for all layers
    # Uncomment and modify these examples to use variable parameters:
    # (0, 10): (180.0, 400.0),      # Layers 0-9: lower power/speed
    # (10, 20): (220.0, 600.0),     # Layers 10-19: higher power/speed
    # (20, 30): (200.0, 500.0),     # Layers 20-29: back to default
    # 50: (250.0, 700.0),           # Layer 50 only: very high power/speed
}

print("📋 Hatching Configuration:")
for key, value in HATCHING_CONFIG.items():
    print(f"   {key}: {value}")

print("\n⚡ Variable Laser Parameters (Discrete per Layer):")
print("   Format: {layer_range: (power W, speed mm/s)}")
for key, value in LAYER_PARAMETERS.items():
    if key == 'default':
        print(f"   {key}: Power={value[0]} W, Speed={value[1]} mm/s")
    elif isinstance(key, tuple):
        print(f"   Layers {key[0]}-{key[1]}: Power={value[0]} W, Speed={value[1]} mm/s")
    else:
        print(f"   Layer {key}: Power={value[0]} W, Speed={value[1]} mm/s")

class STLHatchingVisualizer:
    """
    PyVista-based visualization for STL models with hatching paths.
    
    Features:
    - STL model 3D visualization
    - Hatching path generation and visualization
    - Laser parameter color-coding (Power, Velocity, Energy)
    - 4-section dashboard layout
    - Export to .slm capability
    """
    
    def __init__(self, stl_part):
        self.stl_part = stl_part
        self.layers = []
        self.models = []
        self.build_styles = {}
        
        # Create output widgets for each section
        self.output1 = widgets.Output() if WIDGETS_AVAILABLE else None
        self.output2 = widgets.Output() if WIDGETS_AVAILABLE else None
        self.output3 = widgets.Output() if WIDGETS_AVAILABLE else None
        self.output4 = widgets.Output() if WIDGETS_AVAILABLE else None
        
        # Store generated layers and models
        self.generated_layers = []
        self.generated_models = []
        self.generated_build_styles = {}
    
    def _get_layer_parameters(self, layer_idx, layer_params_config):
        """
        Get laser parameters (power, speed) for a specific layer index.
        
        Args:
            layer_idx: Layer index (0-based)
            layer_params_config: Dictionary mapping layer ranges to (power, speed) tuples
        
        Returns:
            (power, speed) tuple
        """
        default_power, default_speed = layer_params_config.get('default', (200.0, 500.0))
        
        # Check for exact match
        if layer_idx in layer_params_config:
            return layer_params_config[layer_idx]
        
        # Check for range matches
        for key, value in layer_params_config.items():
            if key == 'default':
                continue
            if isinstance(key, tuple):
                start, end = key
                if start <= layer_idx < end:
                    return value
        
        return (default_power, default_speed)
    
    def generate_hatching(self, layer_thickness=None, hatch_angle=None, 
                          volume_offset=None, spot_compensation=None,
                          num_inner_contours=None, num_outer_contours=None,
                          stripe_width=None, layer_params_config=None,
                          layer_start=0, layer_end=None, layer_step=1):
        """
        Generate hatching for the STL model with variable laser parameters per layer.
        
        Args:
            layer_params_config: Dictionary mapping layer ranges to (power, speed) tuples.
                Format: {layer_range: (power, speed), ...}
                - 'default': (power, speed) for all layers not specified
                - int: single layer index
                - tuple (start, end): layer range [start, end)
        
        Returns:
            layers, models, build_styles
        """
        if not PYSLM_AVAILABLE or self.stl_part is None:
            return [], [], {}
        
        # Use defaults if not provided
        default_config = {
            'layer_thickness': 0.04,
            'hatch_angle': 10,
            'volume_offset': 0.08,
            'spot_compensation': 0.06,
            'num_inner_contours': 2,
            'num_outer_contours': 1,
            'stripe_width': 5.0
        }
        
        layer_thickness = layer_thickness if layer_thickness is not None else default_config['layer_thickness']
        hatch_angle = hatch_angle if hatch_angle is not None else default_config['hatch_angle']
        volume_offset = volume_offset if volume_offset is not None else default_config['volume_offset']
        spot_compensation = spot_compensation if spot_compensation is not None else default_config['spot_compensation']
        num_inner_contours = num_inner_contours if num_inner_contours is not None else default_config['num_inner_contours']
        num_outer_contours = num_outer_contours if num_outer_contours is not None else default_config['num_outer_contours']
        stripe_width = stripe_width if stripe_width is not None else default_config['stripe_width']
        
        # Use provided layer_params_config or global LAYER_PARAMETERS
        if layer_params_config is None:
            layer_params_config = LAYER_PARAMETERS
        
        # Create hatcher
        myHatcher = hatching.StripeHatcher()
        myHatcher.stripeWidth = stripe_width
        myHatcher.hatchAngle = hatch_angle
        myHatcher.volumeOffsetHatch = volume_offset
        myHatcher.spotCompensation = spot_compensation
        myHatcher.numInnerContours = num_inner_contours
        myHatcher.numOuterContours = num_outer_contours
        myHatcher.hatchSortMethod = hatching.AlternateSort()
        
        # Create model
        model = pyslm.geometry.Model()
        model.mid = 1
        model.name = "STL_Model"
        
        # Dictionary to store build styles: {(power, speed): build_style}
        build_styles_dict = {}
        build_styles = {}
        next_bid = 1
        
        # Generate layers
        layers = []
        layerId = 0
        
        z_max = self.stl_part.boundingBox[5]
        if layer_end is None:
            layer_end = int(z_max / layer_thickness)
        
        print(f"🔄 Generating hatching with variable parameters...")
        print(f"   Total layers to generate: {min(layer_end, int(z_max / layer_thickness)) - layer_start}")
        
        for i in range(layer_start, min(layer_end, int(z_max / layer_thickness)), layer_step):
            z = i * layer_thickness
            
            # Get laser parameters for this layer
            power, speed = self._get_layer_parameters(i, layer_params_config)
            
            # Get or create build style for this (power, speed) combination
            param_key = (power, speed)
            if param_key not in build_styles_dict:
                buildStyle = pyslm.geometry.BuildStyle()
                buildStyle.bid = next_bid
                buildStyle.laserPower = power
                buildStyle.laserSpeed = speed
                buildStyle.laserFocus = 0.0
                buildStyle.laserId = 1
                buildStyle.laserMode = pyslm.geometry.LaserMode.Pulse
                buildStyle.pointDistance = 50
                buildStyle.pointExposureTime = 80
                
                model.buildStyles.append(buildStyle)
                build_styles_dict[param_key] = buildStyle
                build_styles[buildStyle.bid] = buildStyle
                next_bid += 1
            else:
                buildStyle = build_styles_dict[param_key]
            
            # Rotate hatch angle per layer (common practice)
            myHatcher.hatchAngle = hatch_angle + (i * 66.7)
            
            # Slice the STL
            geomSlice = self.stl_part.getVectorSlice(z)
            
            if len(geomSlice) == 0:
                continue
            
            # Generate hatching
            layer = myHatcher.hatch(geomSlice)
            
            # Assign model and build style to this layer
            for geo in layer.geometry:
                geo.mid = model.mid
                geo.bid = buildStyle.bid
            
            layer.z = int(z * 1000)  # Convert to microns
            layer.layerId = layerId
            layers.append(layer)
            layerId += 1
        
        model.topLayerId = layerId - 1
        
        # Store for later use
        self.generated_layers = layers
        self.generated_models = [model]
        self.generated_build_styles = build_styles
        
        print(f"✅ Generated {len(layers)} layers with {len(build_styles)} different build styles")
        print(f"   Build styles:")
        for bid, bs in build_styles.items():
            print(f"      BID {bid}: Power={bs.laserPower} W, Speed={bs.laserSpeed} mm/s")
        
        return layers, [model], build_styles
    
    def stl_to_pyvista(self):
        """Convert pyslm Part to PyVista mesh with proper transformation."""
        if not PYVISTA_AVAILABLE or self.stl_part is None:
            return None
        
        try:
            # Ensure geometry is regenerated (applies transformations)
            # The geometry property automatically applies transformations via regenerate()
            if hasattr(self.stl_part, 'geometry') and self.stl_part.geometry is not None:
                trimesh_mesh = self.stl_part.geometry
                
                # Get vertices (these should already have transformations applied)
                # The geometry property calls regenerate() which applies getTransform()
                vertices = trimesh_mesh.vertices.copy()
                
                # Convert trimesh to PyVista
                faces = trimesh_mesh.faces
                
                # PyVista expects faces in format: [n, v1, v2, v3, ...]
                faces_pv = np.column_stack([
                    np.full(len(faces), 3),  # Triangle faces
                    faces
                ]).flatten()
                
                mesh = pv.PolyData(vertices, faces_pv)
                return mesh
        except Exception as e:
            # Only print errors, not debug info
            print(f"Error converting STL to PyVista: {e}")
            import traceback
            traceback.print_exc()
        
        return None
    
    def layer_to_pyvista(self, layer, parameter='power', z_pos_mm=None):
        """Convert a pyslm Layer to PyVista polylines."""
        if not PYVISTA_AVAILABLE:
            return []
        
        if z_pos_mm is None:
            z_pos_mm = float(layer.z) / 1000.0
        
        polylines = []
        
        for geom in layer.geometry:
            try:
                if not hasattr(geom, 'coords') or len(geom.coords) < 2:
                    continue
                
                coords = geom.coords
                
                # Get parameter value
                build_style = self.generated_build_styles.get(geom.bid)
                if build_style:
                    if parameter == 'power':
                        param_value = build_style.laserPower
                    elif parameter == 'velocity':
                        param_value = build_style.laserSpeed
                    elif parameter == 'energy':
                        param_value = build_style.laserPower / build_style.laserSpeed if build_style.laserSpeed > 0 else 0
                    else:
                        param_value = 0
                else:
                    param_value = 0
                
                # Convert to 3D
                if coords.shape[1] == 2:
                    points_3d = np.column_stack([
                        coords[:, 0],
                        coords[:, 1],
                        np.full(len(coords), z_pos_mm)
                    ])
                else:
                    points_3d = coords
                
                # Create polyline
                polyline = pv.PolyData(points_3d)
                polyline.lines = np.hstack([
                    [len(points_3d)],
                    np.arange(len(points_3d))
                ])
                
                polyline['parameter'] = np.full(len(points_3d), param_value)
                polylines.append(polyline)
                
            except Exception as e:
                continue
        
        return polylines
    
    # ========================================================================
    # SECTION 1: 3D Plot of STL + All Hatching Layers
    # ========================================================================
    def visualize_stl_all_layers_3d(self, parameter='power', layer_start=0, layer_end=None,
                                    layer_step=1, show_stl=True, show_hatching=True, show_thermal=False):
        """Section 1: 3D visualization of STL model with all hatching layers."""
        if not PYVISTA_AVAILABLE:
            return
        
        if not self.generated_layers:
            print("⚠️ No hatching generated. Please generate hatching first.")
            return
        
        selected_layers = self.generated_layers[layer_start:layer_end:layer_step] if layer_end else self.generated_layers[layer_start::layer_step]
        
        with self.output1:
            clear_output(wait=True)
            
            # Show progress message while processing
            mode = "🌡️ Thermal Image" if show_thermal else "📊 Normal Hatching" if show_hatching else "📦 STL Only"
            print(f"📊 Processing visualization ({mode})...")
            
            plotter = pv.Plotter(notebook=True)
            
            # Add STL mesh
            if show_stl:
                stl_mesh = self.stl_to_pyvista()
                if stl_mesh is not None:
                    plotter.add_mesh(
                        stl_mesh,
                        color='lightgray',
                        opacity=0.3,
                        show_edges=False
                    )
            
            # Show hatching paths (normal mode)
            if show_hatching and not show_thermal:
                all_polylines = []
                param_values = []
                
                for layer in selected_layers:
                    z_pos_mm = float(layer.z) / 1000
                    polylines = self.layer_to_pyvista(layer, parameter, z_pos_mm=z_pos_mm)
                    all_polylines.extend(polylines)
                    for pl in polylines:
                        if 'parameter' in pl.point_data:
                            param_values.extend(pl.point_data['parameter'])
                
                if all_polylines:
                    combined = all_polylines[0]
                    for pl in all_polylines[1:]:
                        combined = combined + pl
                    
                    plotter.add_mesh(
                        combined,
                        scalars='parameter',
                        cmap='plasma',  # Default colormap
                        line_width=2,
                        show_scalar_bar=True,
                        scalar_bar_args={'title': parameter.title()}
                    )
            
            # Show thermal image (thermal mode - overrides hatching if both are checked)
            if show_thermal:
                all_polylines = []
                param_values = []
                
                for layer in selected_layers:
                    z_pos_mm = float(layer.z) / 1000
                    polylines = self.layer_to_pyvista(layer, parameter, z_pos_mm=z_pos_mm)
                    all_polylines.extend(polylines)
                    for pl in polylines:
                        if 'parameter' in pl.point_data:
                            param_values.extend(pl.point_data['parameter'])
                
                if all_polylines:
                    combined = all_polylines[0]
                    for pl in all_polylines[1:]:
                        combined = combined + pl
                    
                    # Use thermal colormap for temperature/energy visualization
                    # 'hot': black -> red -> yellow (classic thermal imaging)
                    # 'inferno': black -> purple -> yellow (perceptually uniform)
                    # 'coolwarm': blue -> white -> red (diverging)
                    colormap = 'hot'  # PyVista thermal colormap
                    line_width = 4  # Thicker for thermal visualization
                    scalar_title = f'🌡️ Thermal: {parameter.title()}' if parameter != 'energy' else f'🌡️ Temperature / Energy'
                    
                    plotter.add_mesh(
                        combined,
                        scalars='parameter',
                        cmap=colormap,
                        line_width=line_width,
                        show_scalar_bar=True,
                        scalar_bar_args={
                            'title': scalar_title,
                            'n_labels': 5,
                            'font_size': 10
                        }
                    )
            
            param_labels = {
                'power': 'Laser Power (W)',
                'velocity': 'Scan Velocity (mm/s)',
                'energy': 'Energy Density (J/mm²)'
            }
            
            # Update title based on what's visible
            visibility = []
            if show_stl:
                visibility.append("STL")
            if show_hatching:
                visibility.append("Hatching")
            if show_thermal:
                visibility.append("Thermal")
            title_parts = " + ".join(visibility) if visibility else "Empty View"
            
            plotter.add_text(
                f'Section 1: {title_parts} - {param_labels.get(parameter, parameter)}',
                font_size=12
            )
            
            plotter.add_axes()
            # Clear progress message and show visualization
            clear_output(wait=True)
            plotter.show(jupyter_backend='static')
    
    # ========================================================================
    # SECTION 2: 3D Plot of STL + Single Layer Hatching
    # ========================================================================
    def visualize_stl_single_layer_3d(self, layer_idx=0, parameter='power', 
                                      elevation=30, azimuth=45, show_stl=True):
        """Section 2: 3D visualization of STL with single layer hatching."""
        if not PYVISTA_AVAILABLE:
            return
        
        if not self.generated_layers or layer_idx >= len(self.generated_layers):
            print("⚠️ Invalid layer index or no hatching generated.")
            return
        
        layer = self.generated_layers[layer_idx]
        
        # Get actual laser parameters for this layer
        layer_params = {}
        if layer.geometry:
            first_geom = layer.geometry[0]
            build_style = self.generated_build_styles.get(first_geom.bid)
            if build_style:
                layer_params = {
                    'power': build_style.laserPower,
                    'velocity': build_style.laserSpeed,
                    'energy': build_style.laserPower / build_style.laserSpeed if build_style.laserSpeed > 0 else 0
                }
        
        with self.output2:
            clear_output(wait=True)
            
            # Show progress while processing
            print(f"📊 Processing layer {layer_idx}...")
            
            plotter = pv.Plotter(notebook=True)
            
            # Add STL mesh
            if show_stl:
                stl_mesh = self.stl_to_pyvista()
                if stl_mesh is not None:
                    plotter.add_mesh(
                        stl_mesh,
                        color='lightgray',
                        opacity=0.3,
                        show_edges=False
                    )
            
            # Add single layer hatching
            polylines = self.layer_to_pyvista(layer, parameter, z_pos_mm=float(layer.z)/1000)
            
            if polylines:
                combined = polylines[0]
                for pl in polylines[1:]:
                    combined = combined + pl
                
                plotter.add_mesh(
                    combined,
                    scalars='parameter',
                    cmap='plasma',
                    line_width=3,
                    show_scalar_bar=True,
                    scalar_bar_args={'title': parameter.title()}
                )
            
            param_labels = {
                'power': 'Laser Power (W)',
                'velocity': 'Scan Velocity (mm/s)',
                'energy': 'Energy Density (J/mm²)'
            }
            
            z_height = float(layer.z) / 1000
            plotter.add_text(
                f'Section 2: STL + Layer {layer_idx} (Z={z_height:.2f} mm) - {param_labels.get(parameter, parameter)}',
                font_size=12
            )
            
            plotter.camera_position = [
                (1, 1, 1),  # Position
                (0, 0, 0),  # Focal point
                (0, 0, 1)   # Up vector
            ]
            plotter.camera.azimuth = azimuth
            plotter.camera.elevation = elevation
            
            plotter.add_axes()
            # Clear progress message and show visualization
            clear_output(wait=True)
            plotter.show(jupyter_backend='static')
    
    # ========================================================================
    # SECTION 3: 2D Layer Cross-Section (STL slice + hatching)
    # ========================================================================
    def visualize_layer_2d(self, layer_idx=0, parameter='power'):
        """Section 3: 2D visualization of single layer (STL slice + hatching)."""
        if not PYVISTA_AVAILABLE:
            return
        
        if not self.generated_layers or layer_idx >= len(self.generated_layers):
            print("⚠️ Invalid layer index.")
            return
        
        layer = self.generated_layers[layer_idx]
        z_height = float(layer.z) / 1000
        
        with self.output3:
            clear_output(wait=True)
            
            # Show progress while processing
            print(f"📐 Processing 2D layer {layer_idx}...")
            
            # Create 2D plotter (orthographic view from top)
            plotter = pv.Plotter(notebook=True)
            
            # Get STL slice at this Z height
            try:
                geom_slice = self.stl_part.getVectorSlice(z_height)
                if geom_slice:
                    # Convert shapely polygons to PyVista
                    for polygon in geom_slice:
                        if hasattr(polygon, 'exterior'):
                            coords = np.array(polygon.exterior.coords)
                            if len(coords) > 2:
                                # Create 2D polyline (Z = layer height)
                                points_2d = np.column_stack([
                                    coords[:, 0],
                                    coords[:, 1],
                                    np.full(len(coords), z_height)
                                ])
                                
                                polyline = pv.PolyData(points_2d)
                                polyline.lines = np.hstack([
                                    [len(points_2d)],
                                    np.arange(len(points_2d))
                                ])
                                
                                plotter.add_mesh(
                                    polyline,
                                    color='black',
                                    line_width=3,
                                    label='STL Boundary'
                                )
            except Exception as e:
                pass  # Silently skip if STL slice can't be rendered
            
            # Add hatching paths
            polylines = self.layer_to_pyvista(layer, parameter, z_pos_mm=z_height)
            
            if polylines:
                combined = polylines[0]
                for pl in polylines[1:]:
                    combined = combined + pl
                
                plotter.add_mesh(
                    combined,
                    scalars='parameter',
                    cmap='plasma',
                    line_width=2,
                    show_scalar_bar=True,
                    scalar_bar_args={'title': parameter.title()}
                )
            
            # Set camera to top view
            plotter.camera_position = 'xy'
            plotter.camera.zoom(1.2)
            
            param_labels = {
                'power': 'Laser Power (W)',
                'velocity': 'Scan Velocity (mm/s)',
                'energy': 'Energy Density (J/mm²)'
            }
            
            plotter.add_text(
                f'Section 3: Layer {layer_idx} (Z={z_height:.2f} mm) - {param_labels.get(parameter, parameter)}',
                font_size=12
            )
            
            plotter.add_axes()
            # Clear progress message and show visualization
            clear_output(wait=True)
            plotter.show(jupyter_backend='static')
    
    # ========================================================================
    # SECTION 4: Statistics Dashboard
    # ========================================================================
    def generate_statistics(self):
        """Section 4: Generate comprehensive statistics."""
        if self.stl_part is None:
            return
        
        with self.output4:
            clear_output(wait=True)
            
            print("=" * 70)
            print("SECTION 4: STL + HATCHING STATISTICS DASHBOARD")
            print("=" * 70)
            print()
            
            # STL Statistics
            print("📦 STL MODEL INFORMATION:")
            print("-" * 70)
            print(f"   Model Name: {self.stl_part.name if hasattr(self.stl_part, 'name') else 'N/A'}")
            print(f"   Bounding Box: {self.stl_part.boundingBox}")
            print(f"   Origin: {self.stl_part.origin}")
            print(f"   Rotation: {self.stl_part.rotation}")
            
            if hasattr(self.stl_part, 'geometry') and self.stl_part.geometry is not None:
                mesh = self.stl_part.geometry
                print(f"   Vertices: {len(mesh.vertices)}")
                print(f"   Faces: {len(mesh.faces)}")
                print(f"   Volume: {mesh.volume:.2f} mm³")
                print(f"   Surface Area: {mesh.area:.2f} mm²")
            print()
            
            # Hatching Statistics
            if self.generated_layers:
                print("🔄 HATCHING INFORMATION:")
                print("-" * 70)
                print(f"   Total Layers: {len(self.generated_layers)}")
                
                if self.generated_layers:
                    z_heights = [float(layer.z) / 1000 for layer in self.generated_layers]
                    print(f"   Build Height: {max(z_heights) - min(z_heights):.2f} mm")
                    print(f"   Z Range: {min(z_heights):.2f} - {max(z_heights):.2f} mm")
                
                # Count geometries
                total_hatches = 0
                total_contours = 0
                for layer in self.generated_layers:
                    for geom in layer.geometry:
                        if hasattr(geom, 'coords'):
                            if len(geom.coords) > 2:
                                total_hatches += 1
                            else:
                                total_contours += 1
                
                print(f"   Total Hatch Geometries: {total_hatches}")
                print(f"   Total Contour Geometries: {total_contours}")
                print()
                
                # Build Style Information
                if self.generated_build_styles:
                    print("⚙️ LASER PARAMETERS (Variable per Layer):")
                    print("-" * 70)
                    print(f"   Total Build Styles: {len(self.generated_build_styles)}")
                    print()
                    
                    # Count layers per build style
                    layers_per_style = {}
                    for i, layer in enumerate(self.generated_layers):
                        if layer.geometry:
                            bid = layer.geometry[0].bid
                            if bid not in layers_per_style:
                                layers_per_style[bid] = []
                            layers_per_style[bid].append(i)
                    
                    for bid, bs in sorted(self.generated_build_styles.items()):
                        layer_indices = layers_per_style.get(bid, [])
                        if len(layer_indices) == 1:
                            layer_str = f"Layer {layer_indices[0]}"
                        elif len(layer_indices) <= 5:
                            layer_str = f"Layers {layer_indices}"
                        else:
                            layer_str = f"Layers {layer_indices[0]}-{layer_indices[-1]} ({len(layer_indices)} layers)"
                        
                        print(f"   Build Style ID {bid}: {layer_str}")
                        print(f"      Laser Power: {bs.laserPower} W")
                        print(f"      Laser Speed: {bs.laserSpeed} mm/s")
                        if bs.laserSpeed > 0:
                            energy = bs.laserPower / bs.laserSpeed
                            print(f"      Energy Density: {energy:.3f} J/mm²")
                        print()
            else:
                print("⚠️ No hatching generated yet.")
                print("   Generate hatching using the widgets above.")
                print()
            
            print("=" * 70)
    
    # ========================================================================
    # WIDGET CREATION
    # ========================================================================
    def create_widgets(self):
        """Create widgets for all 4 sections."""
        if not WIDGETS_AVAILABLE:
            print("⚠️ ipywidgets not available.")
            return
        
        if self.stl_part is None:
            print("⚠️ No STL part loaded.")
            return
        
        # ====================================================================
        # STEP 1: CREATE WIDGETS WITH PLACEHOLDER VALUES (for immediate display)
        # ====================================================================
        # Calculate expected layer count for widget ranges
        z_max = self.stl_part.boundingBox[5]
        try:
            layer_thickness = HATCHING_CONFIG.get('layer_thickness', 0.04)
        except NameError:
            layer_thickness = 0.04
        estimated_layers = min(50, int(z_max / layer_thickness))  # Estimate for initial display
        
        # Show loading messages in outputs immediately
        with self.output1:
            print("⏳ Generating hatching paths... Please wait...")
        with self.output2:
            print("⏳ Generating hatching paths... Please wait...")
        with self.output3:
            print("⏳ Generating hatching paths... Please wait...")
        with self.output4:
            print("⏳ Generating statistics... Please wait...")
        
        # ====================================================================
        # SECTION 1 WIDGETS
        # ====================================================================
        s1_param = widgets.Dropdown(
            options=[
                ('🔴 Laser Power', 'power'), 
                ('🔵 Scan Velocity', 'velocity'), 
                ('⚡ Energy Density', 'energy')
            ],
            value='power',
            description='Parameter:',
            style={'description_width': 'initial'}
        )
        s1_layer_start = widgets.IntSlider(value=0, min=0, max=estimated_layers-1, step=1, description='Start:')
        s1_layer_end = widgets.IntSlider(value=min(20, estimated_layers), min=1, max=estimated_layers, step=1, description='End:')
        s1_layer_step = widgets.IntSlider(value=1, min=1, max=10, step=1, description='Step:')
        s1_show_stl = widgets.Checkbox(value=True, description='Show STL Model')
        s1_show_hatching = widgets.Checkbox(value=True, description='Show Hatching')
        s1_show_thermal = widgets.Checkbox(value=False, description='Show Thermal Image')
        
        def update_s1(change):
            self.visualize_stl_all_layers_3d(
                parameter=s1_param.value,
                layer_start=s1_layer_start.value,
                layer_end=s1_layer_end.value,
                layer_step=s1_layer_step.value,
                show_stl=s1_show_stl.value,
                show_hatching=s1_show_hatching.value,
                show_thermal=s1_show_thermal.value
            )
        
        s1_param.observe(update_s1, names='value')
        s1_layer_start.observe(update_s1, names='value')
        s1_layer_end.observe(update_s1, names='value')
        s1_layer_step.observe(update_s1, names='value')
        s1_show_stl.observe(update_s1, names='value')
        s1_show_hatching.observe(update_s1, names='value')
        s1_show_thermal.observe(update_s1, names='value')
        
        section1 = widgets.VBox([
            widgets.HTML("<h3>📊 Section 1: STL + All Hatching Layers</h3>"),
            s1_param,
            widgets.HBox([s1_layer_start, s1_layer_end, s1_layer_step]),
            widgets.HBox([s1_show_stl, s1_show_hatching, s1_show_thermal]),
            widgets.HTML("<p><small>🌡️ <b>Thermal Image:</b> Uses PyVista 'hot' colormap to simulate thermal imaging. Future: can use actual thermal sensor data.</small></p>"),
            self.output1
        ], layout=widgets.Layout(width='48%', border='1px solid #ccc', padding='10px', margin='5px'))
        
        # ====================================================================
        # SECTION 2 WIDGETS
        # ====================================================================
        s2_param = widgets.Dropdown(
            options=[
                ('🔴 Laser Power', 'power'), 
                ('🔵 Scan Velocity', 'velocity'), 
                ('⚡ Energy Density', 'energy')
            ],
            value='power',
            description='Parameter:',
            style={'description_width': 'initial'}
        )
        s2_layer = widgets.IntSlider(value=0, min=0, max=estimated_layers-1, step=1, description='Layer:')
        s2_elevation = widgets.IntSlider(value=30, min=-90, max=90, step=5, description='Elevation:')
        s2_azimuth = widgets.IntSlider(value=45, min=0, max=360, step=5, description='Azimuth:')
        s2_show_stl = widgets.Checkbox(value=True, description='Show STL')
        
        def update_s2(change):
            self.visualize_stl_single_layer_3d(
                layer_idx=s2_layer.value,
                parameter=s2_param.value,
                elevation=s2_elevation.value,
                azimuth=s2_azimuth.value,
                show_stl=s2_show_stl.value
            )
        
        s2_param.observe(update_s2, names='value')
        s2_layer.observe(update_s2, names='value')
        s2_elevation.observe(update_s2, names='value')
        s2_azimuth.observe(update_s2, names='value')
        s2_show_stl.observe(update_s2, names='value')
        
        section2 = widgets.VBox([
            widgets.HTML("<h3>🎯 Section 2: STL + Single Layer Hatching</h3>"),
            widgets.HBox([s2_param, s2_layer]),
            widgets.HBox([s2_elevation, s2_azimuth]),
            s2_show_stl,
            self.output2
        ], layout=widgets.Layout(width='48%', border='1px solid #ccc', padding='10px', margin='5px'))
        
        # ====================================================================
        # SECTION 3 WIDGETS
        # ====================================================================
        s3_param = widgets.Dropdown(
            options=[
                ('🔴 Laser Power', 'power'), 
                ('🔵 Scan Velocity', 'velocity'), 
                ('⚡ Energy Density', 'energy')
            ],
            value='power',
            description='Parameter:',
            style={'description_width': 'initial'}
        )
        s3_layer = widgets.IntSlider(value=0, min=0, max=estimated_layers-1, step=1, description='Layer:')
        
        def update_s3(change):
            self.visualize_layer_2d(
                layer_idx=s3_layer.value,
                parameter=s3_param.value
            )
        
        s3_param.observe(update_s3, names='value')
        s3_layer.observe(update_s3, names='value')
        
        section3 = widgets.VBox([
            widgets.HTML("<h3>📐 Section 3: 2D Layer Cross-Section</h3>"),
            widgets.HTML("<p><small>STL slice boundary + hatching paths</small></p>"),
            widgets.HBox([s3_param, s3_layer]),
            self.output3
        ], layout=widgets.Layout(width='48%', border='1px solid #ccc', padding='10px', margin='5px'))
        
        # ====================================================================
        # SECTION 4 WIDGETS
        # ====================================================================
        section4 = widgets.VBox([
            widgets.HTML("<h3>📈 Section 4: Statistics Dashboard</h3>"),
            widgets.Button(description='Refresh Statistics', button_style='info'),
            self.output4
        ], layout=widgets.Layout(width='48%', border='1px solid #ccc', padding='10px', margin='5px'))
        section4.children[1].on_click(lambda b: self.generate_statistics())
        
        # ====================================================================
        # STEP 2: DISPLAY DASHBOARD IMMEDIATELY (before hatching generation)
        # ====================================================================
        top_row = widgets.HBox([section1, section2], layout=widgets.Layout(width='100%', justify_content='space-between'))
        bottom_row = widgets.HBox([section3, section4], layout=widgets.Layout(width='100%', justify_content='space-between'))
        
        # Create dashboard container
        dashboard = widgets.VBox([
            widgets.HTML("<h1>🎨 STL + Hatching Visualization Dashboard</h1>"),
            widgets.HTML("<p><b>2x2 Grid Layout:</b> STL model with generated hatching paths</p>"),
            widgets.HTML("<hr>"),
            top_row,
            bottom_row
        ])
        
        # DISPLAY DASHBOARD IMMEDIATELY - This is critical!
        print("\n" + "="*70)
        print("📊 DISPLAYING INTERACTIVE DASHBOARD")
        print("="*70)
        display(dashboard)
        print("✅ Dashboard displayed! Generating hatching paths in background...")
        print()
        
        # ====================================================================
        # STEP 3: GENERATE HATCHING (happens after dashboard is displayed)
        # ====================================================================
        print("🔄 Generating hatching paths (limited to 50 layers for initial visualization)...")
        print("   💡 Tip: You can regenerate with more layers later if needed.")
        
        total_layers = int(z_max / layer_thickness)
        initial_layer_end = min(50, total_layers)
        
        self.generate_hatching(layer_end=initial_layer_end)
        
        if not self.generated_layers:
            print("⚠️ Could not generate hatching. Check STL file and parameters.")
            return
        
        print(f"✅ Generated {len(self.generated_layers)} hatching layers (out of {total_layers} total)")
        print()
        
        # ====================================================================
        # STEP 4: UPDATE WIDGET RANGES WITH ACTUAL VALUES
        # ====================================================================
        actual_max = len(self.generated_layers) - 1
        s1_layer_start.max = actual_max
        s1_layer_end.max = len(self.generated_layers)
        s1_layer_end.value = min(20, len(self.generated_layers))
        s2_layer.max = actual_max
        s3_layer.max = actual_max
        
        # ====================================================================
        # STEP 5: GENERATE INITIAL VISUALIZATIONS
        # ====================================================================
        print("🔄 Generating initial visualizations...")
        try:
            update_s1(None)
            update_s2(None)
            update_s3(None)
            self.generate_statistics()
            print("✅ All visualizations ready!")
        except Exception as e:
            print(f"⚠️ Error generating initial visualizations: {e}")
            import traceback
            traceback.print_exc()


# Create and display dashboard
if stl_part is not None:
    visualizer = STLHatchingVisualizer(stl_part)
    visualizer.create_widgets()
else:
    print("⚠️ Please load an STL file first.")


📋 Hatching Configuration:
   layer_thickness: 0.04
   hatch_angle: 10
   volume_offset: 0.08
   spot_compensation: 0.06
   num_inner_contours: 2
   num_outer_contours: 1
   stripe_width: 5.0

⚡ Variable Laser Parameters (Discrete per Layer):
   Format: {layer_range: (power W, speed mm/s)}
   default: Power=200.0 W, Speed=500.0 mm/s

📊 DISPLAYING INTERACTIVE DASHBOARD


✅ Dashboard displayed! Generating hatching paths in background...

🔄 Generating hatching paths (limited to 50 layers for initial visualization)...
   💡 Tip: You can regenerate with more layers later if needed.
🔄 Generating hatching with variable parameters...
   Total layers to generate: 50
✅ Generated 50 layers with 1 different build styles
   Build styles:
      BID 1: Power=200.0 W, Speed=500.0 mm/s
✅ Generated 50 hatching layers (out of 1025 total)

🔄 Generating initial visualizations...
✅ All visualizations ready!


## 3. STL + Hatching Visualization Class

This class combines STL model visualization with hatching paths in PyVista.

### 4 Sections:
1. **Section 1**: STL + All Hatching Layers (3D) - View all layers with STL mesh
2. **Section 2**: STL + Single Layer Hatching (3D with rotation) - Interactive 3D view of one layer
3. **Section 3**: 2D Layer Cross-Section (STL slice + hatching) - Top-down view of layer
4. **Section 4**: Statistics Dashboard (STL info + hatching stats) - Comprehensive statistics

**Run the cell below to create and display the interactive dashboard!**


## 4. Optional: Export to .slm File

Export the generated hatching to a .slm build file using libSLM.


In [ ]:
# Export to .slm file (optional)
if LIBSLM_AVAILABLE and 'visualizer' in locals() and visualizer.generated_layers:
    export_path = "output_hatching.slm"
    
    try:
        writer = slmsol.Writer()
        writer.setFilePath(export_path)
        
        # Add models
        for model in visualizer.generated_models:
            writer.addModel(model)
        
        # Add layers
        for layer in visualizer.generated_layers:
            writer.addLayer(layer)
        
        # Write file
        writer.write()
        
        print(f"✅ Exported to: {export_path}")
        print(f"   - Models: {len(visualizer.generated_models)}")
        print(f"   - Layers: {len(visualizer.generated_layers)}")
        
    except Exception as e:
        print(f"❌ Error exporting: {e}")
        import traceback
        traceback.print_exc()
else:
    if not LIBSLM_AVAILABLE:
        print("⚠️ libSLM not available. Cannot export to .slm file.")
    elif 'visualizer' not in locals():
        print("⚠️ Visualizer not created. Run the visualization cell first.")
    else:
        print("⚠️ No hatching generated. Generate hatching first.")


❌ Error exporting: 'libSLM.translators.slmsol.Writer' object has no attribute 'addModel'
File 'output_hatching.slm' is ready to write


Traceback (most recent call last):
  File "/tmp/ipykernel_20793/1155386354.py", line 11, in <module>
    writer.addModel(model)
    ^^^^^^^^^^^^^^^
AttributeError: 'libSLM.translators.slmsol.Writer' object has no attribute 'addModel'
